# Workflow diagram

In [ ]:
import pkg_resources
import json
import os
from glob import glob

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [ ]:
PROCESSES = ["1", "2"]

In [ ]:
def read_ipynb(filename):
    fn = pkg_resources.resource_filename("RepoTemp.templates", filename)

    with open(fn, 'r') as f:
        return json.load(f)

In [ ]:
def get_processes(plist):
    node_list = []
    for curr_proc in plist:
        # Find relevant log files
        file_lookfor = f"{os.getcwd()}/{curr_proc}.log"
        file_found = len(glob(file_lookfor))==1

        try:
            assert file_found
        except:
            print(f"File {file_lookfor} not found. Process {curr_proc} not performed?")

        if file_found:
            node_list.append(curr_proc)
    
    return node_list

In [ ]:
def create_graph(nodes: list) -> nx.Graph:
    graph = nx.Graph()
    graph.pos = {}
    for i, node in enumerate(nodes):
        graph.pos[i] = (i,0)
        if i+1 < len(nodes):
            graph.add_edge(i, i+1)
    plt.figure(figsize=[10,3])
    plt.axis('off')
    for i, node in enumerate(nodes):
        plt.text(i, -0.025, node, ha='center')

    return graph

In [ ]:
nodes = get_processes(PROCESSES)
workflow = create_graph(nodes)
nx.draw_networkx(workflow, workflow.pos)

print(nodes)